In [1]:
import numpy as np

telementary = 1
runNum = 0      ## Increment to utilise caching

Taking in input shape

# Layer Class

In [2]:
class nlayer:
    id = 0
    shape = 1               ## Defines self dimension (1D)
    input_layers = []        ## store layer pointer
    weights = None          ## assuming all input activations are concatenated (sorted on layer ID).
    bias = np.array([])      ## store self biases
    activationFn = "linear"         ## store self activation function


    ## Caching
    
    #### store last activation as cache to speed up when multiple layers use this layer as input. So this is evaluated only once.
    cachedRun = -2      # runNum when cache was calculated, can be old
    ## cachedRun = -1 & isAdaptive = 0 is for input layers
    cacheValue = None
    
    ## Flag indicating if it was being evaluated.
    #### This can help in case of self loops, when a layer was being evaluated was evaluated again
    #  meaning one of this layer's input_layer has this layer as one of the inputs (called self-loop is a graph).
    #  In this situation, the last cached value of this layer will be returned.
    # this may be used to simulate LSTM Network.
    beingEvaluated = 0  

    ## Error variance
    #### Store absolute sum of errors in terms of array of sum per node in 1D np array
    isAdaptive = 1
    


    ## Methods
    def __init__(self, shape=1, inputLayers=[], isInput=0, setInputValues=[], ActivationFn="linear", isFixed=0) -> None:
        self.shape = shape
        self.activationFn = ActivationFn
        self.bias = np.zeros(shape)
        self.input_layers = []  ## Clearing on reinitializing

        if(isFixed==1):
            self.isAdaptive = 0
        if(isInput):
            self.cachedRun = -1
            self.isAdaptive = 0
            if(len(setInputValues) != 0):
                self.cacheValue = np.array(setInputValues)
        else:
            # generating random weights if given
            if(type(inputLayers) == type([])):
                if(len(inputLayers) != 0):
                    for layer in inputLayers:
                        self.addInputLayer(layer)
            else:
                print("inputLayers should be a List.")
                if(type(inputLayers) == type(nlayer(1))):
                    self.addInputLayer(inputLayers)

    def addInputLayer(self, newInputLayer):
        # check if it doesn't already exists
        for layr in self.input_layers:
            if(newInputLayer == layr):
                print("Layer already exists.")
                return -1

        self.input_layers.append(newInputLayer)
        ## DONE: Generate random weights
        generatedColumn = np.random.rand(self.shape, newInputLayer.shape)
        if(type(self.weights) == type(None)):
            self.weights = generatedColumn
        else:
            self.weights = np.concatenate((self.weights, generatedColumn), axis=1)

    def addWidth_to_Layer(self, addWidth):
        if(addWidth > 0):
            self.shape += addWidth
            self.bias = np.concatenate((self.bias, np.random.rand(addWidth)))
            
            ## generating new row of random weights
            generatedRow = np.random.rand(addWidth, self.weights.shape[1])
            self.weights = np.concatenate((self.weights, generatedRow))
        else:
            print("error, doesn't support decrease.")


    def calcActivationFn(self,rawActivation):
        if(self.activationFn == "linear"):
            return rawActivation

        if(self.activationFn == "relu"):
            return np.maximum(rawActivation, 0)

    def getActivation(self):    ## return np array of activation of current layer
        ## beingEvaluated == 1 means the node was triggered by a loop in the network. Returning last value cached prevents infinite loops.
        if(self.cachedRun == runNum or self.cachedRun == -1 or self.beingEvaluated == 1):   ## if activation was already calculated for this run OR is an input layer
            return(self.cacheValue)
        else:
            ## compiling a numpy array of all activation values listed in input layer. 
            inputArr = np.array([])

            self.beingEvaluated = 1

            for layrIndx in range(len(self.input_layers)):
                inputArr = np.concatenate((inputArr, self.input_layers[layrIndx].getActivation()))


            self.beingEvaluated = 0

            # Checking if shape matches
            if(inputArr.shape[0] > self.weights.shape[1]):
                if(telementary): print("!!!SHAPE MISMATCH!!!")  
                ## Adjust matrix dimension & adding new random weights to match size
                generatedColumn = np.random.rand(self.weights.shape[0], (inputArr.shape[0] - self.weights.shape[1]))
                self.weights = np.concatenate((self.weights, generatedColumn), axis=1)


            elif(inputArr.shape[0] < self.weights.shape[1]):       ## input layer is removed causing weight matrix to be larger than inputs
                print("!! Input Layer was removed. Unstable !!")
                return -1
            
            rawActivation = np.matmul(self.weights, inputArr) + self.bias
            activation = self.calcActivationFn(rawActivation=rawActivation)

            self.cachedRun = runNum
            # self.cacheValue = activation          ## storing a pointer to activation calculated
            self.cacheValue = np.copy(activation)   ## duplicating array

            if(telementary): print("activation =", activation, "& cached")  

            return activation

            

# Network Class

In [20]:
class network:
    input_shape=1  # Currently only 1D
    output_shape=1 # Currently only 1D
 
    input_layer = None      ## Pointer to input nlayer
    output_layer = None     ## Pointer to output nlayer

    layers = []
    numberOfLayers = 0      ## used to assign ID to new layer in matrix

    adaptive = 1

    def __init__(self, input_shape, output_shape) -> None:
        self.input_shape = input_shape
        self.output_shape = output_shape

        # Connect output with 1 adaptive neuron input
        self.input_layer = nlayer(input_shape, isInput=1)

        hiddenLayer = nlayer(1,inputLayers=[self.input_layer],ActivationFn="relu")

        self.output_layer = nlayer(output_shape, inputLayers=[hiddenLayer], isFixed=1)

    def setInput(self, input_values):
        # print("SETTING INPUT LAYER & STORING VALUES")
        if(type(self.input_layer) != type(None)):
            if(len(input_values) < self.input_layer.shape):
                print("ERROR: Unable to reduce input layer shape. Insert len(input values) >= input_shape")
            else:
                self.input_layer.shape = len(input_values)
                self.input_layer.cachedRun = -1
                self.input_layer.isAdaptive = 0
                self.input_layer.cacheValue = np.array(input_values)
        else:   ## Initialize new input layer
                self.input_layer = nlayer(len(input_values), isInput=1, setInputValues=np.array(input_values))

                
                linker = self.output_layer
                if(type(linker) != type(None)):
                    while(len(linker.input_layers) > 0):    ## following only oldest (1st in list) links to reach input
                        linker = linker.input_layers[0]                               
                    linker.input_layer = [self.input_layer]



    def forward_prop(self, input_values=None):    # find result activation from input activation and weights
        if(type(input_values) != type(None)):
            self.input_layer.cacheValue = input_values

        if(self.input_layer.cachedRun == -1 and type(self.input_layer.cacheValue) != type(None)):
            output_activations = self.output_layer.getActivation()
            return output_activations
        else:
            print("Input uninitialized")
            return -1



# TESTING

In [21]:
n1 = network(3,1)

In [22]:
n1.setInput([3,2,1])

In [23]:
n1.output_layer.input_layers[0].input_layers[0].cacheValue

array([3, 2, 1])

In [24]:
print(n1.output_layer.input_layers[0].cachedRun)

-2


In [25]:
n1.setInput([3,1,2])

In [27]:
print(n1.forward_prop())

[0.2491882]


layerTesting

In [11]:
inLay = nlayer(2, isInput=1, setInputValues=[2,3])

hidLay = nlayer(1, inputLayers=[inLay], ActivationFn="relu")

In [12]:
# hidLay.bias
# hidLay.weights
# hidLay.cacheValue
hidLay.input_layers
# inLay.input_layers

In [8]:
hidLay.getActivation()

activation = [4.91548855] & cached


array([4.91548855])

In [9]:
newInLay = nlayer(3,isInput=1,setInputValues=[2,3,5])

In [10]:
hidLay.addInputLayer(newInputLayer=newInLay)

In [11]:
hidLay.weights

array([[0.99885793, 0.9725909 , 0.72486948, 0.35375928, 0.07909738]])

In [12]:
hidLay.addWidth_to_Layer(2)

In [13]:
hidLay.weights

array([[0.99885793, 0.9725909 , 0.72486948, 0.35375928, 0.07909738],
       [0.10659773, 0.55853347, 0.62134468, 0.07597125, 0.98782191],
       [0.06992685, 0.24441412, 0.73520054, 0.26422487, 0.89743243]])

In [14]:
runNum += 1

In [28]:
hidLay.getActivation()

array([5.99525239, 6.29679149, 7.71707944])

# Playground

In [95]:
# empArr = np.empty(1,dtype=float)
empArr = np.array([4,],dtype=float)
inArr = np.array([1,2],dtype=float)
# inArr = np.

In [96]:
print(inArr)

[1. 2.]


In [109]:
empArr = np.concatenate((empArr, inArr))
print(empArr)
print(empArr.shape)
# print(newArr)


[7. 1. 2. 1. 2.]
(5,)


In [123]:
wts = np.array([[3,2,1],[1,0,-1]],dtype=float)

In [107]:
print("wts", wts.shape)
print("emp", empArr.shape)
act = np.matmul(wts,empArr)
print("act", act)

wts (2, 3)
emp (3,)
act [25.  5.]


In [110]:
empArr

array([7., 1., 2., 1., 2.])

In [117]:
# columns to add = empArr.shape[0] - wts.shape[1]
newCols = np.random.rand(wts.shape[0], empArr.shape[0] - wts.shape[1])

In [124]:
wts

array([[ 3.,  2.,  1.],
       [ 1.,  0., -1.]])

In [126]:
wts = np.concatenate((wts, newCols), axis=1)

In [94]:
np.random.rand(2,3)

array([[0.21734973, 0.22785161, 0.77439285],
       [0.26069918, 0.76749742, 0.33375012]])

In [30]:
x = np.array([1,2])

In [32]:
y = np.array(x)

In [163]:
x = [2,3]
y = x.append(5)

In [164]:
print(x)

[2, 3, 5]


# MAIN

In [5]:
n1 = network(2,1)

in1 = np.array([0,1,2])
wtMat = np.array([[5,6,7],[8,9,10]])
# biases = np.array([5,25])
biases = np.array([0.5,0.25])

In [37]:
output_activations = np.matmul(wtMat, in1) + biases
print(output_activations)

[20.5  29.25]


In [38]:
n1.forward_prop(input_activations=in1, weight_matrix=wtMat, bias_ndarrray=biases)

array([20.5 , 29.25])